In [ ]:
import sys
sys.path.append('../common')
import utils
import importlib
importlib.reload(utils)

from utils import IOHandler as IO

import numpy as np
import time
import sys
import os

import pynput.keyboard as kb
import pynput.mouse as m

from collections import defaultdict
from enum import Enum

io = IO(offset=(1081, 675), game_dims=(1200, 900), verbose=True)

In [ ]:
class Screen(Enum):
    GAME_OVER = "game_over"
    ARCADE = "arcade"
    HOME = "home"
    GREAT_PLAY = "great_play"
    SCORE_EXCEEDS = "score_exceeds"
    MIDGAME = "midgame"
    RECESS = "recess"

SCREEN_TYPES = {
    Screen.GAME_OVER: [680, 380, [246, 196, 195]], # piggy bank
    Screen.ARCADE: [170, 852, [143, 60, 148]], # magenta arcade cabinet
    Screen.HOME: [600, 30, [235, 52, 35]], # red R in logo
    Screen.GREAT_PLAY: [290, 670, [172, 68, 59]], # dark red gift box
    Screen.SCORE_EXCEEDS: [330, 465, [36, 0, 141]], # webkinz logo outline
    Screen.MIDGAME: [1090, 65, [179, 108, 247]], # music note
    Screen.RECESS: [610, 37, [23, 46, 138]]
}

# Given masks, bounding box, game image, number of digits, and whether centered, parse digits in the image
def parse_digits(img, dims, pos, num_digits, masks, centered = True):
    w,h = dims
    x,y = pos

    if img is None:
        num_img = io.capture_portion(x, y, w*num_digits, h)
    else:
        num_img = img[x:x+w*num_digits,y:y+h,:]

    guess_1, error_1 = _parse_digits(num_img, num_digits, masks, dims)

    if not centered:
        return guess_1

    num_digits -= 1
    num_img = num_img[w//2:-w//2,:,:]

    guess_2, error_2 = _parse_digits(num_img, num_digits, masks, dims)

    if error_1 < error_2:
        return guess_1
    else:
        return guess_2

# Helper to parse digits
def _parse_digits(num_img, num_digits, masks, dims):
    w,h = dims
    digits = []
    total_error = 0
    for i in range(num_digits):
        mask_name, error = io.get_best_mask_dict(num_img[w*i:w*(i+1)], masks)
        if mask_name.isdigit():
            digit = int(mask_name)
        elif mask_name == 'B':
            digit = 0
        elif mask_name == 'blank':
            digit = None
        digits.append(digit)
        total_error += error
    digits = [x for x in digits if x is not None]
    guess = int('0'+''.join(map(str, digits)))
    return guess, total_error

# Parse the level number
def get_level(img = None):
    dims = 20,30
    num_digits = 3
    masks = score_masks
    centered = True

    if is_screen_type(Screen.RECESS, img):
        pos = 445,30
        return -1*parse_digits(img, dims, pos, num_digits, masks, centered)
    else:
        pos = 769,30
        return parse_digits(img, dims, pos, num_digits, masks, centered)

# Parse the time remaining
def get_time(img = None):
    if not is_screen_type(Screen.RECESS, img):
        return None

    dims = 20,30
    pos = 753,30
    num_digits = 3
    masks = score_masks
    centered = True

    return parse_digits(img, dims, pos, num_digits, masks, centered)

# Parse the kinzcash
def get_money(img = None):
    dims = 20,30
    pos = 972,30
    num_digits = 4
    masks = score_masks
    centered = True

    if is_screen_type(Screen.RECESS, img):
        pos = 970,30

    return parse_digits(img, dims, pos, num_digits, masks, centered)

# Parse the sum goal
def get_sum_goal(img = None):
    dims = 48,70
    pos = 804,173
    num_digits = 3
    masks = sum_masks
    centered = True

    return parse_digits(img, dims, pos, num_digits, masks, centered)

# Parse the sum goal, after it stops animating
def get_sum_goal_wait(img = None, sec = 0.05):
    dims = 48,70
    pos = 804,173
    num_digits = 3
    masks = sum_masks
    centered = True

    prev = None
    if img is None:
        while True:
            smol = io.capture_portion(pos[0],pos[1],dims[0]*num_digits,dims[1]*num_digits)
            if (smol == prev).all():
                break
            prev = smol
            time.sleep(sec)

    return parse_digits(img, dims, pos, num_digits, masks, centered)

# Parse the number of sums left
def get_sums_left(img = None):
    if is_screen_type(Screen.RECESS, img):
        return None
    
    dims = 20,30
    pos = 536,30
    num_digits = 3
    masks = score_masks
    centered = True

    return parse_digits(img, dims, pos, num_digits, masks, centered)

# Parse the score
def get_score(img = None):
    dims = 20,30
    pos = 142,30
    num_digits = 7
    masks = score_masks
    centered = True

    return parse_digits(img, dims, pos, num_digits, masks, centered)

# Click the tile at the given index
def click_tile(idx):
    corner = np.array([40,116])
    dims = np.array([62,62])
    offset = np.array([8,8])
    pos = corner + 0.5*dims + (dims+offset)*idx
    io.click_mouse(pos[0], pos[1], delays=[0.03, 0.03, 0.03])

# Decode the tiles of the board
def decode_board(img=None):
    if img is None:
        img = io.capture_screen()
    board = np.empty(shape=[10, 11], dtype=object)
    for i in range(10):
        for j in range(11):
            tile = decode_tile(img=img, idx=np.array([i,j]))
            board[i,j] = tile
    return board

# Return the board's image and non-floating tiles
def capture_decode_board_wait(sec=0.01):
    x,y = 40,116
    w,h = 62,62
    a,b = 8,8
    M,N = 10,11

    prev_board = None
    while True:
        time.sleep(sec)
        board_img = io.capture_screen()
        board = remove_floating(decode_board(board_img))
        if (board == prev_board).all():
            break
        prev_board = board
    return board_img, board

# Deselect any selected tiles
def deselect_selected(board):
    for i in range(0, board.shape[0]):
        for j in range(0, board.shape[1]):
            tile = board[i,j]
            if "true" in tile:
                click_tile(np.array([i,j]))
            board[i][j] = tile.replace("true", "false")
    return board

# Pretty print the board
def print_board(board):
    global STICKER_FGS
    global TILE_COLORS
    global TILE_SYM

    output = ""
    for j in range(0, board.shape[1]):
        for i in range(0, board.shape[0]):
            tile = board[i,j]
            if tile not in SYMBOLS:
                number, sticker, selected = tile.split("_")
                if selected.lower() == "true":
                    bg = TILE_COLORS["selected"]["bg_code"]
                else:
                    if number.isdigit():
                        num = int(number) % 10
                        if num == 0:
                            num = 10
                        bg = TILE_COLORS[str(num)]["bg_code"]
                    else:
                        bg = TILE_COLORS[number]["bg_code"]
                fg = STICKER_FGS[sticker]["code"]
                sym = TILE_SYM[number]
                SYMBOLS[tile] = f"\033[38;5;{fg}m\033[48;5;{bg}m{sym}\033[0m"
            output += SYMBOLS[tile]
        output += "\n"
    print(output)
    print()

# Test the number parser (helpful after changes).
def test_number_parser():
    digit_tests = [
        { 'path': 'no-commit/saved/2021-09-05 19.41.00.png', 'expected': {'score': 4, 'sums': 19, 'level': 1, 'money': 0, 'sum': 4, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.41.07.png', 'expected': {'score': 8, 'sums': 18, 'level': 1, 'money': 0, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.41.12.png', 'expected': {'score': 12, 'sums': 17, 'level': 1, 'money': 0, 'sum': 7, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.41.18.png', 'expected': {'score': 18, 'sums': 16, 'level': 1, 'money': 1, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-07 11.29.03.png', 'expected': {'score': 260, 'sums': None, 'level': -1, 'money': 17, 'sum': 7, 'time': 10} },
        { 'path': 'no-commit/saved/2021-09-05 19.41.23.png', 'expected': {'score': 22, 'sums': 15, 'level': 1, 'money': 1, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.41.37.png', 'expected': {'score': 38, 'sums': 11, 'level': 1, 'money': 2, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.41.45.png', 'expected': {'score': 50, 'sums': 9, 'level': 1, 'money': 3, 'sum': 3, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.41.52.png', 'expected': {'score': 56, 'sums': 7, 'level': 1, 'money': 3, 'sum': 7, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.42.04.png', 'expected': {'score': 72, 'sums': 3, 'level': 1, 'money': 4, 'sum': 4, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.42.15.png', 'expected': {'score': 72, 'sums': 3, 'level': 1, 'money': 4, 'sum': 4, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.42.31.png', 'expected': {'score': 84, 'sums': 18, 'level': 2, 'money': 5, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.42.36.png', 'expected': {'score': 84, 'sums': 18, 'level': 2, 'money': 5, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.42.49.png', 'expected': {'score': 91, 'sums': 17, 'level': 2, 'money': 6, 'sum': 7, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.43.08.png', 'expected': {'score': 126, 'sums': 9, 'level': 2, 'money': 8, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.43.15.png', 'expected': {'score': 131, 'sums': 8, 'level': 2, 'money': 8, 'sum': 4, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.43.41.png', 'expected': {'score': 154, 'sums': 20, 'level': 3, 'money': 10, 'sum': 5, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.43.48.png', 'expected': {'score': 158, 'sums': 19, 'level': 3, 'money': 10, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.43.54.png', 'expected': {'score': 158, 'sums': 19, 'level': 3, 'money': 10, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.44.01.png', 'expected': {'score': 164, 'sums': 18, 'level': 3, 'money': 10, 'sum': 8, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 19.44.21.png', 'expected': {'score': 168, 'sums': 17, 'level': 3, 'money': 11, 'sum': 8, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 20.50.24.png', 'expected': {'score': 463, 'sums': 20, 'level': 5, 'money': 30, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 20.52.03.png', 'expected': {'score': 780, 'sums': 20, 'level': 7, 'money': 52, 'sum': 9, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-05 20.52.12.png', 'expected': {'score': 795, 'sums': 17, 'level': 7, 'money': 53, 'sum': 10, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-06 10.26.40.png', 'expected': {'score': 0, 'sums': 20, 'level': 1, 'money': 0, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-06 10.26.44.png', 'expected': {'score': 0, 'sums': 20, 'level': 1, 'money': 0, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-06 10.27.02.png', 'expected': {'score': 34, 'sums': 14, 'level': 1, 'money': 2, 'sum': 4, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-06 10.27.22.png', 'expected': {'score': 18, 'sums': 16, 'level': 1, 'money': 1, 'sum': 4, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-06 10.27.45.png', 'expected': {'score': 31, 'sums': 15, 'level': 1, 'money': 2, 'sum': 4, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-06 10.27.53.png', 'expected': {'score': 41, 'sums': 13, 'level': 1, 'money': 2, 'sum': 4, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-06 10.28.13.png', 'expected': {'score': 28, 'sums': 15, 'level': 1, 'money': 1, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-06 10.28.18.png', 'expected': {'score': 28, 'sums': 15, 'level': 1, 'money': 1, 'sum': 6, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-06 13.35.38.png', 'expected': {'score': 0, 'sums': 20, 'level': 1, 'money': 0, 'sum': 3, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-06 13.35.47.png', 'expected': {'score': 0, 'sums': 20, 'level': 1, 'money': 0, 'sum': 0, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-06 13.56.36.png', 'expected': {'score': 1293, 'sums': 20, 'level': 9, 'money': 96, 'sum': 11, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-07 11.27.09.png', 'expected': {'score': 0, 'sums': 20, 'level': 1, 'money': 0, 'sum': 3, 'time': None} },
        { 'path': 'no-commit/saved/2021-09-07 11.29.03.png', 'expected': {'score': 260, 'sums': None, 'level': -1, 'money': 17, 'sum': 7, 'time': 10} }
    ]

    for test in digit_tests:
        img = io.load_image(test['path'])
        actual = {'score': get_score(img), 'sums': get_sums_left(img), 'level': get_level(img), 'money': get_money(img), 'sum': get_sum_goal(img), 'time': get_time(img)}
        if actual != test['expected']:
            print(f"Failed test #{i} ({test['path']}):\n\tACTUAL:\t\t{actual}\n\tEXPECTED:\t{test['expected']}")

# Determine the type of sticker in the tile image
def decode_sticker(tile_img):
    if io.does_pixel_equal(39, 13, [132,148,68], image=tile_img) and io.does_pixel_equal(33, 53, [196,41,43], image=tile_img):
        name = "apple"
    elif io.does_pixel_equal(12, 39, [244,189,193], image=tile_img) and io.does_pixel_equal(52, 24, [237,112,120], image=tile_img):
        name = "eraser"
    elif io.does_pixel_equal(31, 10, [100,24,145], image=tile_img) and io.does_pixel_equal(31, 50, [65,141,172], image=tile_img):
        name = "clock"
    else:
        name = "none"
    return name

def decode_type(tile):
    outline_colors = {name: colors['outline'] for name, colors in TILE_COLORS.items() if 'outline' in colors}

    outline_pix = tile[30,1]
    best_name, best_dist = IO.get_best_color_dict(outline_pix, outline_colors)
    color_1 = best_name

    outline_pix = tile[30,60]
    best_name, best_dist = IO.get_best_color_dict(outline_pix, outline_colors)
    color_2 = best_name

    if "banner" in [color_1, color_2]:
        return "blank"

    if (color_1 == "10" or color_1 == "blank") and (color_2 == "10" or color_2 == "blank"):
        if IO.same_colors(tile[14,22], np.array([255]*3)):
            return "10"

    if color_1 == color_2:
        if color_1 != "blank" and IO.same_colors(tile[14,22], np.array([255]*3)):
            return int(color_1) + 10
        return color_1
    else:
        return "blank"

# Decode the game image and tile index to {name}_{sticker}_{selected} format
def decode_tile(img = None, idx = None):
    if img is None:
        img = io.capture_screen()
    if idx is None:
        idx = np.array([0,0])
    x,y = 40,116
    w,h = 62,62
    a,b = 8,8
    M,N = 10,11
    tile = img[x+idx[0]*(w+a):x+idx[0]*(w+a)+w, y+idx[1]*(h+b):y+idx[1]*(h+b)+h, :]
    name = decode_type(tile)
    selected = IO.same_colors(tile[10,10,:], TILE_COLORS["selected"]["fill"])
    sticker = decode_sticker(tile)
    if name == "blank":
        selected = False
        sticker = "none"
    return f"{name}_{sticker}_{str(selected).lower()}"

# Return the board with floating tiles removed
def remove_floating(board):
    for i in range(board.shape[0]):
        for j in range(board.shape[1]-2,-1,-1):
            if "blank" in board[i][j+1]:
                name, sticker, selected = board[i][j].split("_")
                board[i][j] = f"blank_none_false"
    return board

# Return the priority of a given tile based on it's position, sticker, number, and whether selected
def tile_priority(tile):
    name, sticker, selected, col, top_idx = tile.split("_")
    if name == "blank":
        name = "100"
    return int(top_idx), sticker == "none", int(name), bool(selected), col

def get_tile_freqs(board):
    tile_freqs = defaultdict(int)
    for i in range(board.shape[0]):
        top_idx = None
        for j in range(board.shape[1]):
            tile = board[i][j]
            if "blank" not in tile:
                if top_idx is None:
                    top_idx = j
                tile += f"_{i}_{top_idx}"
                tile_freqs[tile] += 1
    tile_freqs = {key: tile_freqs[key] for key in sorted(tile_freqs.keys(), key=tile_priority)}
    return tile_freqs

# Returns a map of tile type to the number of it that we should use
# e.g. {'1_none_false_5_4': 1, '1_none_false_1_4': 1, '2_eraser_false_6_4': 1, '1_apple_false_9_4': 1}
def pick_tiles(tile_freqs, goal):
    if goal < 0:
        return False, {}
    if goal == 0:
        return True, defaultdict(int)
    for i, tile_freq in enumerate(tile_freqs.items()):
        tile,freq = tile_freq
        if freq == 0:
            continue
        name, sticker, selected, col, top_idx = tile.split("_")
        val = int(name)

        new_freqs = defaultdict(int)
        for j, new_freq in enumerate(tile_freqs.items()):
            tile_2,freq_2 = new_freq
            name_2,sticker_2,selected_2,col_2,top_idx_2 = tile_2.split("_")
            if j >= i and freq_2 > 0:
                if col_2 == col:
                    top_idx_2 = str(int(top_idx_2) + 1)
                if tile_2 == tile:
                    new_freqs["_".join([name_2,sticker_2,selected_2,col_2,top_idx_2])] = freq_2 - 1
                else:
                    new_freqs["_".join([name_2,sticker_2,selected_2,col_2,top_idx_2])] = freq_2
        new_freqs = {key: new_freqs[key] for key in sorted(new_freqs.keys(), key=tile_priority)}

        is_possible, ans = pick_tiles(new_freqs, goal - val)
        if is_possible:
            ans[tile] += 1
            return is_possible, ans
    return False, {}

# Returns the indices of tiles on the board that match the desired type distribution
def find_best_tiles(picked, board):
    new_picked = defaultdict(int)
    for k,v in picked.items():
        name, sticker, selected, col, top_idx = k.split("_")
        new_picked["_".join([name, sticker, selected, col])] += v
    tiles = []
    for j in range(board.shape[1]):
        for i in range(board.shape[0]):
            tile = board[i][j]
            tile += f"_{i}"
            if tile in new_picked:
                new_picked[tile] -= 1
                if new_picked[tile] == 0:
                    new_picked.pop(tile)
                tiles.append((i,j))
    return tiles

# Click each of the tiles in the list
def submit_sum(tiles):
    for tile in tiles:
        click_tile(tile)

# Calculates the y-idx of the highest tile in the board
def calc_max_height(board):
    for j in range(board.shape[1]):
        for i in range(board.shape[0]):
            tile = board[i][j]
            if "blank" not in tile:
                return board.shape[1] - j
    return 0

# TODO: move break/kill program to utils and make name consistent in bots   
# TODO: use tilda
def on_press(key):
    global break_program, listener
    if key == kb.Key.esc:
        print("escape pressed")
        break_program = True
        listener.stop()
        return False
    elif key == kb.Key.tab:
        print("tab pressed")
        io.save_game_capture()

# TODO: Move screen type and get into game logic to utils
def is_screen_type(screen_type, image=None):
    x, y, color = SCREEN_TYPES[screen_type]
    return io.is_given_screen(x, y, color, delta=10, image=image)

def calc_screen_types(image=None):
    return {screen_type for screen_type in SCREEN_TYPES.keys() if is_screen_type(screen_type, image)}

# Get into the main game
def get_into_game(game_mode = 'level'):
  global break_program
  while True:
    if break_program:
      break
    game_img = io.capture_screen()
    screen_types = calc_screen_types(game_img)
    print(screen_types)

    if Screen.SCORE_EXCEEDS in screen_types:
        io.click_mouse(670, 520)
    elif Screen.GREAT_PLAY in screen_types:
        io.click_mouse(470, 980)
    elif Screen.GAME_OVER in screen_types:
        io.click_mouse(250, 700)
    elif Screen.HOME in screen_types:
        io.click_mouse(180, 850)
    elif Screen.MIDGAME in screen_types:
        return screen_types
    time.sleep(0.2)


In [ ]:
verbose = len(sys.argv) > 1 and sys.argv[1] == "-v"

if os.name == "nt":
    PLATFORM = "windows"
elif os.name == "posix":
    PLATFORM = "macOS"

EMOJIS = None
SYMBOLS = None
BACKGROUNDS = None

keyboard = kb.Controller()
mouse = m.Controller()

game_start = None
SYMBOLS = {}

TILE_SYM = {f"{i}": f"{i:>2}" for i in range(1,100)}
TILE_SYM["blank"] = "  "
TILE_SYM["banner"] = "  "

STICKER_FGS = {
    "clock": {"code": 27},
    "eraser": {"code": 213},
    "apple": {"code": 124},
    "none": {"code": 15}
}

# Note: 8+ are estimates
TILE_COLORS = {
    "selected": {"fill": np.array([153,153,153]), "bg_code": 246},
    "blank": {"fill": np.array([39,92,23]), "outline": np.array([39,92,23]), "bg_code": 22},
    "banner": {"fill": np.array([73,162,248]), "outline": np.array([43,104,168]), "bg_code": 39},
    "1": {"fill": np.array([254,245,81]), "outline": np.array([230,214,71]), "bg_code": 220},
    "2": {"fill": np.array([237,123,47]), "outline": np.array([218,103,41]), "bg_code": 202},
    "3": {"fill": np.array([235,52,35]), "outline": np.array([147,27,19]), "bg_code": 196},
    "4": {"fill": np.array([164,31,22]), "outline": np.array([95,16,10]), "bg_code": 88},
    "5": {"fill": np.array([222,58,246]), "outline": np.array([130,29,148]), "bg_code": 201},
    "6": {"fill": np.array([125,33,186]), "outline": np.array([73,19,82]), "bg_code": 91},
    "7": {"fill": np.array([89,199,250]), "outline": np.array([65,147,179]), "bg_code": 45},
    "8": {"fill": np.array([8,23,149]), "outline": np.array([1,4,73]), "bg_code": 19},
    "9": {"fill": np.array([108,229,70]), "outline": np.array([56,129,35]), "bg_code": 76},
    "10": {"fill": np.array([64,139,39]), "outline": np.array([39,94,24]), "bg_code": 28},
}

sum_masks = io.load_images_from_directory('sum')
sum_masks = {name.replace('mask_', '').replace('.png', ''): mask for name, mask in sum_masks.items()}

score_masks = io.load_images_from_directory('score')
score_masks = {name.replace('mask_', '').replace('.png', ''): mask for name, mask in score_masks.items()}

break_program = False

listener = kb.Listener(on_press=on_press)
listener.start()

io.click_mouse(0,0)

known_goal = None

while not break_program:
    screen_types = get_into_game()
    is_recess_level = Screen.RECESS in screen_types
    
    game_img, board = capture_decode_board_wait()
    board = deselect_selected(board)
    print_board(board)

    if not is_recess_level:
        max_height = calc_max_height(board)
        if max_height < 9 - get_level() // 2.5:
            io.click_mouse(700, 145, delays=[0.03, 0.03, 0]) # Add a row of tiles

    tile_freqs = get_tile_freqs(board)

    if not is_recess_level:
        goal = get_sum_goal_wait()
        known_goal = None
    elif known_goal is None or known_goal == 0:
        goal = get_sum_goal_wait()
        known_goal = goal

    is_possible, picked = pick_tiles(tile_freqs, goal)
    print(f'Goal: {goal}')
    print(f'Picked tile distrib: {picked}')
    if is_possible:
        best = find_best_tiles(picked, board)
        print(f'Best tile indices: {best}')
        submit_sum(best)
    else:
        print("Sum not possible with current board, adding row")
        io.click_mouse(700, 145) # Add a row of tiles
        time.sleep(0.2)